# Creación de la Variable “Diferencia de Tráfico” entre Zonas Dentro y Fuera de la ZBE

## Contexto y Objetivo

Creación de una variable innovadora para el análisis del impacto de las Zonas de Bajas Emisiones (ZBE) en Madrid: la diferencia diaria de tráfico entre estaciones situadas dentro y fuera de la ZBE. Esta variable permitirá analizar si la implantación de la ZBE provoca un desplazamiento del tráfico y, potencialmente, de la contaminación hacia zonas limítrofes.

**Objetivo:**  
Crear una variable diaria que refleje la diferencia de tráfico entre estaciones dentro y fuera de la ZBE, para su uso como variable predictora en modelos de calidad del aire.

In [3]:
# Importaciones básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns  # Comentado por ahora

# Configuración básica de gráficos
plt.style.use('default')  # Usamos el estilo por defecto de matplotlib
# sns.set_palette("husl")  # Comentado por ahora

# Configuración de pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Configuración para ignorar warnings molestos
import warnings
warnings.filterwarnings('ignore')

In [25]:
df_trafico = pd.read_csv('/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/DataSets/Tráfico/trafico_diario_completo.csv', sep=',')
df_trafico.head()

,fecha_solo,intensidad_mean,intensidad_max,vmed_mean,vmed_min
0,2024-01-01,232.110162,6269,6.046687,0.0
1,2024-01-02,323.761436,11244,5.680850,0.0
2,2024-01-03,345.140912,48840,5.616502,0.0
3,2024-01-04,354.376267,54242,5.563690,0.0
4,2024-01-05,345.367104,7280,5.719347,0.0


In [ ]:
import pandas as pd
import glob
import os

# Ruta a la carpeta principal de tráfico
carpeta = '/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/'

# Buscar todos los archivos CSV en todas las subcarpetas
archivos_csv = glob.glob(os.path.join(carpeta, '**', '*.csv'), recursive=True)

print("Archivos encontrados:", len(archivos_csv))
print(archivos_csv[:5])  # Muestra los primeros 5 para comprobar

# Lista para guardar los DataFrames
lista_df = []

# Leer y unir todos los archivos
for archivo in archivos_csv:
    df_temp = pd.read_csv(archivo, sep=';')  # Cambia el separador si es necesario
    lista_df.append(df_temp)

# Unir todos los DataFrames en uno solo
df_trafico_bruto = pd.concat(lista_df, ignore_index=True)

# Mostrar las primeras filas para comprobar
df_trafico_bruto.head()

Archivos encontrados: 36
['/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/trafico_2021/01-2021.csv', '/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/trafico_2021/11-2021.csv', '/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/trafico_2021/10-2021.csv', '/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/trafico_2021/06-2021.csv', '/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/csvs trafico bruto/Trafico/trafico_2021/07-2021.csv']


In [ ]:
# Convertir la columna 'fecha' a tipo datetime y extraer solo la fecha
df_trafico_bruto['fecha'] = pd.to_datetime(df_trafico_bruto['fecha'])
df_trafico_bruto['fecha_solo'] = df_trafico_bruto['fecha'].dt.date

In [ ]:
# Calcular la media diaria de tráfico por estación
df_trafico_diario = df_trafico_bruto.groupby(['id', 'fecha_solo'])['intensidad'].mean().reset_index()
df_trafico_diario = df_trafico_diario.rename(columns={'intensidad': 'intensidad_mean'})
df_trafico_diario.head()

# 1. Identificación y clasificación de estaciones de tráfico según su localización respecto a la ZBE

Para poder calcular la diferencia diaria de tráfico entre zonas dentro y fuera de la ZBE, primero es 
imprescindible identificar y clasificar cada punto de medición según su localización geográfica: si 
está dentro o fuera de la ZBE.

## 1.1. Exploración de los datos de ubicación de estaciones

In [ ]:
# Cargar el fichero de ubicación de estaciones (ajusta la ruta si es necesario)
df_ubicacion = pd.read_csv('/Users/maruxamoreiraperez/Desktop/EscritorioMaru/Cursos/MacAir_Data Science /01_CAPSTONE/DataSets/ubicacion estaciones/pmed_ubicacion_02-2025.csv', sep=';', encoding='latin1')
# Visualizar las primeras filas y las columnas clave
df_ubicacion[['id', 'nombre', 'longitud', 'latitud']].head()

## 1.2. Clasificación de las estaciones como "dentro" o "fuera" de la ZBE

In [ ]:
!conda install shapely -y

In [ ]:
# Definir los polígonos aproximados de las ZBE 
from shapely.geometry import Point, Polygon

# ZBE Centro (coordenadas aproximadas)
zbe_centro_coords = [
    (-3.703790, 40.420000),
    (-3.695000, 40.420000),
    (-3.695000, 40.430000),
    (-3.703790, 40.430000)
]
zbe_centro = Polygon(zbe_centro_coords)

# ZBE Plaza Elíptica (coordenadas aproximadas)
zbe_eliptica_coords = [
    (-3.710000, 40.385000),
    (-3.698000, 40.385000),
    (-3.698000, 40.395000),
    (-3.710000, 40.395000)
]
zbe_eliptica = Polygon(zbe_eliptica_coords)

# Madrid ZBE (zona general, polígono muy amplio y aproximado)
madrid_zbe_coords = [
    (-3.750000, 40.370000),
    (-3.650000, 40.370000),
    (-3.650000, 40.480000),
    (-3.750000, 40.480000)
]
madrid_zbe = Polygon(madrid_zbe_coords)

# Funciones para comprobar si una estación está dentro de cada polígono
def esta_en_zbe_centro(lon, lat):
    return zbe_centro.contains(Point(lon, lat))

def esta_en_zbe_eliptica(lon, lat):
    return zbe_eliptica.contains(Point(lon, lat))

def esta_en_madrid_zbe(lon, lat):
    return madrid_zbe.contains(Point(lon, lat))

# Aplicar las funciones a cada estación
df_ubicacion['en_zbe_centro'] = df_ubicacion.apply(lambda row: esta_en_zbe_centro(row['longitud'], row['latitud']), axis=1)
df_ubicacion['en_zbe_eliptica'] = df_ubicacion.apply(lambda row: esta_en_zbe_eliptica(row['longitud'], row['latitud']), axis=1)
df_ubicacion['en_madrid_zbe'] = df_ubicacion.apply(lambda row: esta_en_madrid_zbe(row['longitud'], row['latitud']), axis=1)

# Visualizar el resultado
df_ubicacion[['id', 'nombre', 'longitud', 'latitud', 'en_zbe_centro', 'en_zbe_eliptica', 'en_madrid_zbe']].head()

## 1.3. Creación de la variable: diferencia diaria de tráfico entre estaciones dentro/fuera de cada ZBE

In [ ]:
# Unimos la información de ubicación (con las columnas de ZBE) al DataFrame de tráfico diario
df_trafico_diario = df_trafico_diario.merge(
    df_ubicacion[['id', 'en_zbe_centro', 'en_zbe_eliptica', 'en_madrid_zbe']],
    on='id', how='left'
)

# Rellenar los NaN de las columnas de ZBE con False
for col in ['en_zbe_centro', 'en_zbe_eliptica', 'en_madrid_zbe']:
    df_trafico_diario[col] = df_trafico_diario[col].fillna(False)

# ¿Hay estaciones dentro de la ZBE en 2024?
print(df_trafico_diario[(df_trafico_diario['en_zbe_centro'] == True) & (df_trafico_diario['fecha_solo'].astype(str).str.startswith('2024'))])

# Función para calcular la diferencia diaria de tráfico para una zona
def calcular_diferencia_trafico(df, columna_zbe):
    trafico_dentro = df[df[columna_zbe]].groupby('fecha_solo')['intensidad_mean'].mean()
    trafico_fuera = df[~df[columna_zbe]].groupby('fecha_solo')['intensidad_mean'].mean()
    df_dif = pd.DataFrame({
        f'trafico_dentro_{columna_zbe}': trafico_dentro,
        f'trafico_fuera_{columna_zbe}': trafico_fuera
    })
    df_dif[f'diferencia_trafico_{columna_zbe}'] = df_dif[f'trafico_dentro_{columna_zbe}'] - df_dif[f'trafico_fuera_{columna_zbe}']
    return df_dif

# Calculamos la variable para cada zona
dif_centro = calcular_diferencia_trafico(df_trafico_diario, 'en_zbe_centro')
dif_eliptica = calcular_diferencia_trafico(df_trafico_diario, 'en_zbe_eliptica')
dif_madrid_zbe = calcular_diferencia_trafico(df_trafico_diario, 'en_madrid_zbe')

# Visualizamos el resultado para la ZBE Centro
dif_centro.head()

## 1.4. Análisis y visualización de la variable “Diferencia de Tráfico”

Análisis y visualización de la evolución temporal de la diferencia diaria de tráfico entre 
estaciones situadas dentro y fuera de la ZBE. El objetivo es identificar patrones, tendencias 
y posibles efectos frontera asociados a la implantación de las Zonas de Bajas Emisiones en Madrid.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
dif_centro['diferencia_trafico_en_zbe_centro'].plot()
plt.title('Diferencia diaria de tráfico (dentro - fuera) en ZBE Centro')
plt.xlabel('Fecha')
plt.ylabel('Diferencia de tráfico')
plt.axhline(0, color='red', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Eje Y (vertical):
Es la diferencia diaria de tráfico entre estaciones “dentro” y “fuera” de la ZBE Centro.
- Si el valor es positivo, significa que el tráfico medio dentro de la ZBE Centro es mayor que fuera.
- Si el valor es negativo, significa que el tráfico medio dentro de la ZBE Centro es menor que fuera.

In [ ]:
# Media móvil de 30 días
plt.figure(figsize=(12, 5))
dif_centro['diferencia_trafico_en_zbe_centro'].rolling(window=30).mean().plot()
plt.title('Diferencia diaria de tráfico (media móvil 30 días) en ZBE Centro')
plt.xlabel('Fecha')
plt.ylabel('Diferencia de tráfico (media móvil)')
plt.axhline(0, color='red', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Resumen mensual
dif_centro.index = pd.to_datetime(dif_centro.index)

plt.figure(figsize=(12, 5))
dif_centro['diferencia_trafico_en_zbe_centro'].resample('M').mean().plot()
plt.title('Diferencia mensual de tráfico (media mensual) en ZBE Centro')
plt.xlabel('Fecha')
plt.ylabel('Diferencia de tráfico (media mensual)')
plt.axhline(0, color='red', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Interpretación de la Evolución de la Diferencia de Tráfico “Dentro vs. Fuera” de las ZBE (2021-2024)

La evolución temporal de la variable “Diferencia de Tráfico” muestra varios patrones y rupturas que pueden explicarse por el contexto normativo y social de Madrid en los últimos años:

- **2020-2021: Impacto de la pandemia COVID-19**
  - Durante 2020 y buena parte de 2021, Madrid (como el resto de España) estuvo sometida a fuertes restricciones de movilidad, incluyendo confinamientos domiciliarios, limitaciones de desplazamiento entre Comunidades Autónomas y toques de queda.
  - Según informes oficiales del Ayuntamiento de Madrid y la DGT, el tráfico rodado descendió drásticamente, llegando a reducirse hasta un 80% en los momentos más estrictos del confinamiento (marzo-mayo 2020) y manteniéndose por debajo de los niveles prepandemia durante meses posteriores.
  - Estas restricciones afectaron tanto a las zonas dentro como fuera de la ZBE, homogeneizando el tráfico y reduciendo la diferencia entre ambas áreas.
  - Referencia: [Informe DGT sobre movilidad 2020](https://www.dgt.es/comunicacion/noticias/2021/Informe-movilidad-2020/)

- **2022: Implementación de nuevas Zonas de Bajas Emisiones**
  - En diciembre de 2021 se puso en marcha la ZBEDEP Plaza Elíptica, y a lo largo de 2022 se amplió la ZBE a todo el municipio de Madrid, restringiendo progresivamente la circulación de vehículos sin distintivo ambiental (etiqueta A).
  - Estas medidas provocaron cambios en los patrones de tráfico, especialmente en los accesos y límites de las nuevas zonas reguladas, lo que se refleja en la gráfica como un aumento puntual de la diferencia de tráfico (más tráfico relativo fuera de la ZBE, posiblemente por efecto frontera o desvíos).
  - Referencia: [Ayuntamiento de Madrid - Zonas de Bajas Emisiones](https://www.madrid.es/portales/munimadrid/es/Inicio/Movilidad-y-transportes/Zonas-de-Bajas-Emisiones/)
 
La variable “Diferencia de Tráfico” refleja fielmente el impacto de las políticas públicas y los eventos excepcionales sobre la movilidad urbana. Los picos y valles observados no solo responden a la implantación de las ZBE, sino también a factores externos como la pandemia de COVID-19 y las restricciones asociadas. Este análisis aporta valor añadido al contextualizar los datos y permite interpretar correctamente los resultados en modelos predictivos de calidad del aire.

# 2. Análisis exploratorio y visualización avanzada de la variable dentro/fuera

## 2.1. Distribución y outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,5))
sns.histplot(dif_centro['diferencia_trafico_en_zbe_centro'], bins=30, kde=True, color='royalblue')
plt.title('Distribución de la diferencia diaria de tráfico (dentro vs. fuera ZBE Centro)')
plt.xlabel('Diferencia de tráfico')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(8,4))
sns.boxplot(x=dif_centro['diferencia_trafico_en_zbe_centro'], color='orange')
plt.title('Boxplot de la diferencia diaria de tráfico')
plt.xlabel('Diferencia de tráfico')
plt.show()

Las gráficas muestran la distribución y los valores atípicos de la variable “diferencia diaria de tráfico” entre estaciones situadas dentro y fuera de la ZBE Centro de Madrid:

- **Distribución**: La variable presenta una distribución aproximadamente normal, aunque ligeramente sesgada hacia valores negativos. Esto indica que, en la mayoría de los días, el tráfico dentro de la ZBE Centro es menor que fuera, lo que es coherente con las restricciones de acceso y circulación impuestas en el área central de Madrid.
- **Valores negativos**: La mayor frecuencia de valores negativos refleja el efecto esperado de las políticas de restricción de tráfico en la ZBE Centro, especialmente tras la implantación de nuevas normativas en 2022.
- **Valores positivos y outliers**: Existen días con valores positivos o extremos, que pueden estar asociados a eventos puntuales (obras, desvíos, festividades, episodios meteorológicos adversos, etc.) o a periodos de transición normativa. Estos outliers son importantes porque pueden indicar situaciones excepcionales que alteran el patrón habitual de movilidad.
- **Contexto temporal**: Entre 2021 y 2024, la movilidad en Madrid ha estado marcada por la recuperación tras la pandemia de COVID-19, la implantación progresiva de las Zonas de Bajas Emisiones y la adaptación de la ciudadanía a las nuevas restricciones. Todo ello se refleja en la variabilidad y dispersión de la variable.

**Conclusión:**  
El análisis de la distribución y los outliers confirma que la variable innovadora es sensible tanto a las políticas públicas como a los eventos excepcionales, y resulta útil para captar los cambios en la movilidad urbana de Madrid en los últimos años.

## 2.2. Análisis de estacionalidad de la diferencia de tráfico
Búscamos si existen patrones estacionales (mensuales, semanales, por días de la semana, etc.) en la diferencia de tráfico entre dentro y fuera de la ZBE Centro.

In [ ]:
# Índice es de tipo fecha
dif_centro.index = pd.to_datetime(dif_centro.index)

# Creamos una columna con el mes y el año
dif_centro['mes_anio'] = dif_centro.index.to_period('M')

# Calculamos la media mensual
media_mensual = dif_centro.groupby('mes_anio')['diferencia_trafico_en_zbe_centro'].mean()

# Gráfica de la media mensual
plt.figure(figsize=(14,5))
media_mensual.plot(marker='o', color='teal')
plt.title('Evolución mensual de la diferencia de tráfico (dentro vs. fuera ZBE Centro)')
plt.xlabel('Mes-Año')
plt.ylabel('Diferencia media de tráfico')
plt.grid(True)
plt.show()

In [ ]:
# Creamos una columna con el día de la semana
dif_centro['dia_semana'] = dif_centro.index.day_name()

# Calculamos la media por día de la semana
media_semana = dif_centro.groupby('dia_semana')['diferencia_trafico_en_zbe_centro'].mean()
# Ordenamos los días para que salgan en orden natural
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
media_semana = media_semana.reindex(dias_orden)

# Gráfica
plt.figure(figsize=(10,5))
media_semana.plot(kind='bar', color='coral')
plt.title('Diferencia media de tráfico por día de la semana')
plt.xlabel('Día de la semana')
plt.ylabel('Diferencia media de tráfico')
plt.xticks(rotation=45)
plt.show()

**Evolución mensual de la diferencia de tráfico (dentro vs. fuera ZBE Centro):**
- Se observa una clara ruptura en la tendencia a partir de principios de 2022, coincidiendo con la implantación de nuevas restricciones y la ampliación de las Zonas de Bajas Emisiones en Madrid.
- Durante 2022, la diferencia de tráfico se vuelve positiva y alcanza sus valores máximos, lo que sugiere un posible efecto frontera: parte del tráfico que antes circulaba por el interior de la ZBE Centro se desplaza a zonas limítrofes.
- A partir de mediados de 2022, la diferencia disminuye y tiende a estabilizarse, reflejando una adaptación progresiva de la movilidad urbana a las nuevas normativas.
- En los años previos (2021) y posteriores (2023), la diferencia de tráfico se mantiene mayoritariamente en valores negativos, lo que indica que, en condiciones normales, el tráfico suele ser mayor dentro de la ZBE Centro que fuera.

**Diferencia media de tráfico por día de la semana:**
- De lunes a viernes, la diferencia de tráfico es negativa y bastante estable, lo que refleja la mayor actividad laboral y de servicios en el centro de Madrid durante los días laborables.
- Los fines de semana (especialmente sábados y domingos), la diferencia se vuelve positiva, indicando que el tráfico fuera de la ZBE Centro supera al del interior. Esto puede deberse a una reducción de la actividad en el centro y a un aumento de desplazamientos hacia zonas periféricas para ocio, compras o actividades familiares.
- Este patrón semanal es coherente con la realidad de Madrid y refuerza la utilidad de la variable para captar dinámicas urbanas y efectos frontera.

**Conclusión:**  
El análisis estacional confirma que la variable “diferencia de tráfico” es sensible tanto a cambios normativos como a patrones de movilidad habituales en Madrid. Su comportamiento la convierte en un excelente indicador para estudios de impacto y modelización de calidad del aire.

## 2.3. Relación entre la variable dentro/fuera de tráfico y la evolución del NO₂

La interpretación se basa en la comparación entre:
- La evolución temporal y estacional de la diferencia de tráfico (dentro vs. fuera de la ZBE Centro), obtenida en nuestro análisis exploratorio.
- Las gráficas multi-anuales de NO₂ por zonas (ZBE Centro, Plaza Elíptica, Castellana, Cuatro Caminos) y el análisis mensual de este contaminante.
- La tabla resumen de diferencias de NO₂ entre días laborables y fines de semana.

Conclusiones clave:
- Los periodos en los que la diferencia de tráfico aumenta (especialmente tras la implantación de nuevas ZBE en 2022) coinciden con descensos o cambios en la tendencia de los niveles de NO₂, especialmente en el interior de la ZBE Centro.
- El patrón semanal de la variable de tráfico (mayor diferencia los fines de semana) se refleja también en los datos de NO₂, donde los valores suelen ser más bajos en fines de semana, confirmando la relación directa entre movilidad y contaminación.
- La comparación multi-anual muestra que, tras la implantación de las ZBE, la brecha entre los valores de NO₂ dentro y fuera de las zonas reguladas tiende a reducirse, lo que sugiere un efecto positivo de las políticas de restricción del tráfico.

En resumen:
La variable de diferencia de tráfico es un excelente indicador para explicar y anticipar la evolución del NO₂ en Madrid, ya que capta tanto los efectos de las políticas públicas como los patrones reales de movilidad observados en los datos.

# 3. Preparación de variable para modelado

In [ ]:
df_variable = dif_centro.reset_index()
print(df_variable.columns)

In [ ]:
# Selección de las columnas necesarias
df_variable = df_variable[['fecha_solo', 'diferencia_trafico_en_zbe_centro']]

# Renombrar la columna a 'fecha' si así la tiene Raquel en su dataset
df_variable = df_variable.rename(columns={'fecha_solo': 'fecha'})

# Guarda el CSV
df_variable.to_csv('variable_diferencia_trafico.csv', index=False)

In [ ]:
import os
print(os.listdir())

In [ ]:
import pandas as pd
pd.read_csv('variable_diferencia_trafico.csv').head()

In [ ]:
from IPython.display import FileLink
FileLink('variable_diferencia_trafico.csv')